In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

24/08/07 10:08:57 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/07 10:08:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 10:08:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/07 10:08:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema= True, header = True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [6]:
data.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [7]:
for item in data.head(2)[1]:
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [10]:
# here we work on numerical data not the first three data for email, address, and avatar
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
# it grabs all numerical columns and convert it into single vectors
assembler = VectorAssembler(inputCols = [ 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent'], outputCol = 'features')

In [13]:
# transform the data
output = assembler.transform (data)

In [15]:
output.printSchema()
# here we see a new columns with name "features"

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [17]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [18]:
final_data = output.select('features', 'Yearly Amount Spent')

In [19]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [20]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [22]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                356|
|   mean| 500.06263577730374|
| stddev|  79.38190036299862|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [23]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                144|
|   mean|  497.4633388373288|
| stddev|  79.39483388931842|
|    min| 256.67058229005585|
|    max|  708.9351848669818|
+-------+-------------------+



In [31]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [32]:
lr_model = lr.fit(train_data)

24/08/07 10:24:04 WARN Instrumentation: [cbe599fb] regParam is zero, which might cause numerical instability and overfitting.


In [33]:
test_results = lr_model.evaluate(test_data)

In [34]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-3.92219590139575...|
|-2.55795384873636...|
|-1.42676981340628...|
|5.684341886080801...|
|2.182787284255027...|
|6.821210263296962...|
|8.185452315956354...|
|2.000888343900442...|
|-2.84217094304040...|
|-5.45696821063756...|
|-2.84217094304040...|
|-1.21644916362129...|
|2.324895831407047...|
|-1.70530256582424...|
|3.410605131648481...|
|1.080024958355352...|
|2.000888343900442...|
|-2.22826201934367...|
|3.751665644813329...|
|-9.94759830064140...|
+--------------------+
only showing top 20 rows



In [35]:
test_results.rootMeanSquaredError

1.529566503614008e-11

In [36]:
test_results.r2

1.0

In [37]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [38]:
unlabeled_data = test_data.select('features')

In [39]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.8364326747734...|
|[31.0472221394875...|
|[31.0613251567161...|
|[31.1239743499119...|
|[31.4252268808548...|
|[31.5261978982398...|
|[31.5702008293202...|
|[31.5761319713222...|
|[31.6610498227460...|
|[31.7216523605090...|
|[31.7366356860502...|
|[31.8164283341993...|
|[31.8512531286083...|
|[31.8627411090001...|
|[31.8745516945853...|
|[31.9048571310136...|
|[31.9262720263601...|
|[31.9453957483445...|
|[31.9480174211613...|
+--------------------+
only showing top 20 rows



In [41]:
predictions = lr_model.transform(unlabeled_data)

In [42]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|282.47124571991844|
|[30.8364326747734...|467.50190042699217|
|[31.0472221394875...|392.49739918903566|
|[31.0613251567161...|487.55545805790155|
|[31.1239743499119...|486.94705383974394|
|[31.4252268808548...| 530.7667186547612|
|[31.5261978982398...| 409.0945261923296|
|[31.5702008293202...| 545.9454921413849|
|[31.5761319713222...| 541.2265839893312|
|[31.6610498227460...| 416.3583535799063|
|[31.7216523605090...| 347.7769266318755|
|[31.7366356860502...| 496.9334462555441|
|[31.8164283341993...| 501.1224915036331|
|[31.8512531286083...|472.99224666681545|
|[31.8627411090001...| 556.2981411740463|
|[31.8745516945853...| 392.2852442462664|
|[31.9048571310136...|473.94985742279613|
|[31.9262720263601...| 392.2049334443487|
|[31.9453957483445...| 657.0199239376482|
|[31.9480174211613...|461.92087689290776|
+--------------------+------------